In [1]:
import pandas as pd
from pandas import Series, DataFrame
import country_converter as coco
import os

In [2]:
path = '/Users/ariana/desktop/historical_tech/raw data/irena'
os.chdir(path)

In [3]:
# https://pxweb.irena.org/pxweb/en/IRENASTAT

In [4]:
irena = os.listdir(path)
for file in irena:
    if '.xlsx' not in file:
        irena.remove(file)
irena

['irena_biogas.xlsx',
 'irena_solar thermal.xlsx',
 'irena_marine.xlsx',
 'irena_offshore on grid.xlsx',
 'irena_liquid biofuel.xlsx',
 'irena_solar pv.xlsx',
 'irena_onshore wind.xlsx',
 'irena_solid biofuels.xlsx',
 'irena_geothermal.xlsx']

In [5]:
def read_irena(file_name):
    file = os.path.abspath(file_name)
    df = pd.read_excel(file, header=2, na_values='..', skipfooter=26)
    df.dropna(how='all', inplace=True)
    df.rename(columns={'Unnamed: 0': 'Country Name'}, inplace=True)
    df['Data Source'] = 'IRENASTAT'
    df['Spatial Scale'] = 'National'
    df['Metric'] = 'Installed electricity capacity'
    df['Unit'] = 'MW'
    tech = df['Unnamed: 1'].iloc[0]
    df['Technology Name'] = tech.title()
    if tech == 'Onshore wind energy':
        country_names = []
        for country in df[df['Unnamed: 2']=='On-grid']['Country Name']:
            country_names.append(country)
        df = df[df['Unnamed: 2']=='on- and off-grid total']
        df['Country Name'] = country_names
    iso2 = []
    for country in df['Country Name']:
        iso2.append(coco.convert(names=country, to='iso2'))
    df['Country Code'] = iso2
    df.drop(columns=['Unnamed: 1', 'Unnamed: 2'], inplace=True)
    df['ID'] = df['Technology Name'] + '_' + df['Metric'] + '_' + df['Country Code']
    df.set_index('ID', inplace=True)
    return df

In [6]:
irena_df = []
for tech in irena:
    df = read_irena(tech)
    irena_df.append(df)

In [7]:
irena_concat = pd.concat(irena_df)

In [8]:
omit = []
for col in irena_concat.columns:
    if len(col) != 4 or col == 'Unit':
        omit.append(col)
empty_rows = irena_concat.drop(columns=omit)
empty_rows.dropna(how='all', inplace=True)
na_idx = []
for country in irena_concat.index:
    if country not in empty_rows.index:
        na_idx.append(country)
irena_concat.drop(na_idx, inplace=True)
irena_concat

,Country Name,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2019,2020,2021,Data Source,Spatial Scale,Metric,Unit,Technology Name,Country Code,2022
ID,,,,,,,,,,,,,,,,,,,,,
Biogas_Installed electricity capacity_AR,Argentina,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,43.675,54.575,68.975,IRENASTAT,National,Installed electricity capacity,MW,Biogas,AR,NaN
Biogas_Installed electricity capacity_AM,Armenia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.300,...,NaN,NaN,NaN,IRENASTAT,National,Installed electricity capacity,MW,Biogas,AM,NaN
Biogas_Installed electricity capacity_AU,Australia,90.0,92.0,99.0,143.0,207.0,265.000,270.000,273.000,218.000,...,190.000,197.000,197.900,IRENASTAT,National,Installed electricity capacity,MW,Biogas,AU,NaN
Biogas_Installed electricity capacity_AT,Austria,24.0,27.0,18.0,29.0,29.0,217.782,222.169,289.352,284.659,...,163.740,164.579,164.579,IRENASTAT,National,Installed electricity capacity,MW,Biogas,AT,NaN
Biogas_Installed electricity capacity_AZ,Azerbaijan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.050,1.050,1.050,IRENASTAT,National,Installed electricity capacity,MW,Biogas,AZ,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Geothermal Energy_Installed electricity capacity_RO,Romania,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.050,0.050,0.050,IRENASTAT,National,Installed electricity capacity,MW,Geothermal Energy,RO,NaN
Geothermal Energy_Installed electricity capacity_RU,Russian Federation,23.0,21.0,70.0,70.0,56.0,79.000,87.000,90.000,80.000,...,81.400,81.400,81.400,IRENASTAT,National,Installed electricity capacity,MW,Geothermal Energy,RU,NaN
Geothermal Energy_Installed electricity capacity_TH,Thailand,0.3,0.3,0.3,0.3,0.3,0.300,0.300,0.300,0.300,...,0.300,0.300,0.300,IRENASTAT,National,Installed electricity capacity,MW,Geothermal Energy,TH,NaN


In [9]:
set(irena_concat['Technology Name'])

{'Biogas',
 'Geothermal Energy',
 'Liquid Biofuels',
 'Marine Energy',
 'Offshore Wind Energy',
 'Onshore Wind Energy',
 'Solar Photovoltaic',
 'Solar Thermal Energy',
 'Solid Biofuels'}

In [10]:
irena_concat.to_csv('/Users/ariana/desktop/historical_tech/cleaned data/irena.csv')